In [1]:
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to work with', ws.name)

Ready to work with dp100-workspace


In [2]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from azureml.core import Dataset,Datastore


data=pd.read_csv('data2/car-prediction.csv')


# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace = ws, name = "car-training3")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# applying label encoding
pre = preprocessing.LabelEncoder()
data = data.apply(pre.fit_transform)

#Normalize the numeric columns
scaler = MinMaxScaler()
num_cols = ['Selling_Price','Present_Price','Kms_Driven','car_age']
data[num_cols] = scaler.fit_transform(data[num_cols])


# Separate features and labels
x, y = data[['Selling_Price','Present_Price','Kms_Driven','Fuel_Type','Seller_Type','Transmission','car_age']].values,data['Owner'].values

# Split data into training set and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = LogisticRegression().fit(x_train, y_train)

# calculate accuracy
y_hat = model.predict(x_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(x_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
model_file = 'car_model2.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
print('Registering model...')
run.register_model(model_path='outputs/car_model2.pkl', model_name='car_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# Get the registered model
model = ws.models['car_model']

print('Model trained and registered.')

Starting experiment: car-training3
Training a decision tree model
Accuracy: 0.9180327868852459
AUC: 0.6499999999999999
Registering model...
Model trained and registered.


In [3]:
# Deploying Model

import os

folder_name = 'car_service'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(experiment_folder, exist_ok=True)

print(folder_name, 'folder created.')

# Set path for scoring script
script_file = os.path.join(experiment_folder,"score_car.py")

car_service folder created.


In [4]:
%%writefile $script_file
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('car_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = json.loads(raw_data)['data']
    np_data = np.array(data)
    # Get a prediction from the model
    predictions = model.predict(np_data)
    # print the data and predictions (so they'll be logged!)
    log_text = 'Data:' + str(data) + ' - Predictions:' + str(predictions)
    print(log_text)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-sold', 'sold']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Overwriting ./car_service/score_car.py


In [5]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

# Save the environment config as a .yml file
env_file = folder_name + "/car_env.yml"
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

Saved dependency info in car_service/car_env.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- scikit-learn
channels:
- anaconda
- conda-forge



In [7]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_file,
                                   conda_file=env_file)

service_name = "car-service-app-insights2"
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
aci_service = Model.deploy(workspace=ws,
                           name= service_name,
                           models= [model],
                           inference_config= inference_config,
                           deployment_config=deployment_config)
aci_service.wait_for_deployment(show_output = True)
print(aci_service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-01-31 17:20:26+00:00 Creating Container Registry if not exists.
2022-01-31 17:20:26+00:00 Registering the environment.
2022-01-31 17:20:29+00:00 Use the existing image.
2022-01-31 17:20:29+00:00 Generating deployment configuration.
2022-01-31 17:20:32+00:00 Submitting deployment to compute..
2022-01-31 17:20:45+00:00 Checking the status of deployment car-service-app-insights2..
2022-01-31 17:26:00+00:00 Checking the status of inference endpoint car-service-app-insights2.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [8]:
# Enable AppInsights
aci_service.update(enable_app_insights=True)
print('AppInsights enabled!')

AppInsights enabled!


In [9]:
endpoint = aci_service.scoring_uri
print(endpoint)

http://8282f107-a551-451b-9338-5cfcf6718f14.eastus.azurecontainer.io/score
